In [1]:
from naginterfaces.library import opt
import numpy as np
from naginterfaces.base import utils

# Matrix completion using Semi-Definite Programming (SDP)

## Correct Rendering of this notebook

This notebook makes use of the `latex_envs` Jupyter extension for equations and references.  If the LaTeX is not rendering properly in your local installation of Jupyter , it may be because you have not installed this extension.  Details at https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/latex_envs/README.html

## Introduction

We start with a survey of $n_r$ respondent with $n_q$ questions. Unfortunately, some of the entries are missing...

Here is an example with $n_r = 15$ and $n_q = 6$
\begin{equation*}
\hat{Y} = 
\begin{bmatrix}
* & * & * & * & * &  0.4\\
0.6 &  0.4 &  0.8 & * & * & *\\
1.0 & * &  0.8 & * &  0.2 & *\\
0.8 &  0.2 & * & * & * & *\\
1.0 &  0.4 & * &  0.0 & * &  0.2\\
0.4 & * & * &  0.2 & * &  0.2\\
1.0 &  0.8 &  0.2 &  0.6 & * & *\\
1.0 & * &  0.2 & * & * & *\\
1.0 &  0.4 & * &  0.6 &  0.0 & *\\
1.0 & * &  0.4 & * & * & *\\
1.0 & * &  0.2 &  0.2 &  0.4 &  0.4\\
1.0 & * & * & * &  1.0 &  0.8\\
1.0 & * &  0.2 & * & * &  0.6\\
1.0 & * & * & * & * &  0.2\\
0.6 & * &  0.2 &  0.4 & * & *\\
\end{bmatrix}
\end{equation*}
What is the best way to fill the missing values? One way to look at it would be to say that respondents with similar answers should stay similar. In practice, this can be modelled as minimizing the rank of $Y$.
\begin{equation*}
\min_Y rank(Y)\\
Y_{ij} = \hat{Y}_{ij}
\end{equation*}
Rank minimization is in general NP-hard but it can be approximated by a heuristic, minimizing the nuclear norm of the matrix. The nuclear norm of a matrix is the sum of its singular values. A rank deficient matrix must have (several) zero singular values. Given the fact that the singular values are always non-negative, a minimization of the nuclear norm has the same effect as $l_1$ norm in compress sensing, i.e., it encourages many singular values to be zero and thus it can be considered as a heuristic for the original rank minimization problem.
\begin{equation*}
\min_Y ||Y||_*\\
Y_{ij} = \hat{Y}_{ij}
\end{equation*}
This can be achieved by solving the following SDP problem:
\begin{equation*}
\min_{Y, W_1, W_2} trace(W_1) + trace(W_2)\\
Y_{ij} = \hat{Y}_{ij}\\
\begin{bmatrix}
W_1 & Y\\
Y^T & W_2
\end{bmatrix} \succeq 0
\end{equation*}

In [2]:
# Start by defining the data
nr = 15
nq = 6
# Matrix, missing values set to-1
Y = np.array([[-1.0,-1.0,-1.0,-1.0,-1.0, 0.4],
                [ 0.6, 0.4, 0.8,-1.0,-1.0,-1.0],
                [-1.0,-1.0, 0.8,-1.0, 0.2,-1.0],
                [ 0.8, 0.2,-1.0,-1.0,-1.0,-1.0],
                [-1.0, 0.4,-1.0, 0.0,-1.0, 0.2],
                [ 0.4,-1.0,-1.0, 0.2,-1.0, 0.2],
                [-1.0, 0.8, 0.2, 0.6,-1.0,-1.0],
                [-1.0,-1.0, 0.2,-1.0,-1.0,-1.0],
                [-1.0, 0.4,-1.0, 0.6, 0.0,-1.0],
                [-1.0,-1.0, 0.4,-1.0,-1.0,-1.0],
                [-1.0,-1.0, 0.2, 0.2, 0.4, 0.4],
                [-1.0,-1.0,-1.0,-1.0, 1.0, 0.8],
                [ 1.0,-1.0, 0.2,-1.0,-1.0, 0.6],
                [-1.0,-1.0,-1.0,-1.0,-1.0, 0.2],
                [ 0.6,-1.0, 0.2, 0.4,-1.0,-1.0]])

# Count the number of missing element
n_miss = 0
for i in range(nr):
    for j in range(nq):
        if Y[i][j] <= -1.0:
            n_miss += 1

We can now start defining the SDP problem. $W_1$ and $W_2$ are full matrices of variables, respectively $n_r \times n_r$ and $n_q \times n_q$. Additionally, the missing values are also be variables of the optimization problem. 

In [3]:
# Number of variables: missing values + 2 full matrix nrxnr and nqxnq 
# (only the upper triangular part is needed)
nvar = int(n_miss + nr*(nr+1)/2 + nq*(nq+1)/2)

# Initialize the problem handle
handle = opt.handle_init(nvar)

# Initialize the linear objective to 0.0
cvec = np.zeros(nvar)

Start defining the linear matrix inequality. First the known values of $\hat{Y}$, stored in the constant part of the matrix inequality.

In [4]:
# Initialize the sparse matrix inequality matrices 
nnzasum = nvar + nr*nq - n_miss
nnza = np.empty(nvar+1, dtype=int)
irowa = np.empty(nnzasum, dtype=int)
icola = np.empty(nnzasum, dtype=int)
a = np.empty(nnzasum, dtype=float)

# number of nonzeros in the constant block
nnza[0] = nr*nq - n_miss

# All the other matrices in the combination only have one element
nnza[1:] = 1

# Fill A_0 with the known values of Y
idx = 0
for i in range(nr):
    for j in range(nq):
        if Y[i][j] >= 0.0:
            irowa[idx] = i + 1
            icola[idx] = nr + j + 1
            a[idx] = -Y[i][j]
            idx += 1

Now fill the upper left block
\begin{bmatrix}
W_1 & * \\
* & *
\end{bmatrix}
consisting of the upper triangular part of $W_1$.

In [5]:
# 1x1 block of the matrix
# Fill the objective vector at the same time
idxobj = 0
for i in range(nr):
    # this is a diagonal element of W1, it belongs to the trace (set objective)
    cvec[idxobj] = 1.0
    for j in range(i, nr):
        irowa[idx] = i + 1
        icola[idx] = j + 1
        a[idx] = 1.0
        idx += 1
        idxobj += 1

Time for the lower right block
\begin{bmatrix}
* & * \\
* & W_2
\end{bmatrix}
consisting of the upper triangular part of $W_2$.

In [6]:
for i in range(nq):
    # this is a diagonal element of W2, it belongs to the trace (set objective)
    cvec[idxobj] = 1.0
    for j in range(i, nq):
        irowa[idx] = nr + i + 1
        icola[idx] = nr + j + 1
        a[idx] = 1.0
        idx += 1
        idxobj += 1

Lastly, the missing elements of $Y$

In [7]:
for i in range(nr):
    for j in range(nq):
        if Y[i][j] < 0.0:
            irowa[idx] = i + 1 
            icola[idx] = nr + j + 1
            a[idx] = 1.0
            idx +=1

Update the problem handle

In [8]:
# add the linear objective to the handle
opt.handle_set_linobj(handle, cvec)

# Add the linear matrix inequalities
dima = nr + nq
idblk = opt.handle_set_linmatineq(handle, dima, nnza, irowa, icola, a, blksizea=None, idblk=0)

# Set optional argument
for option in ['Print Options = No',
               'Initial X = Automatic',
               'Dimacs = Check']:
    opt.handle_opt_set(handle, option)

The problem is ready to be solved

In [9]:
# I/O
iom = utils.FileObjManager(locus_in_output=False)

# Call the solver
x = np.empty(nvar)
inform = 0
x, _, _, _, rinfo, stats, _ = opt.handle_solve_pennon(handle, x, inform, u=None, 
                                                      uc=None, ua=None, io_manager=iom)


 --------------------------------
  E04SV, NLP-SDP Solver (Pennon)
 --------------------------------

 Problem Statistics
   No of variables                196
     bounds               not defined
   No of lin. constraints           0
     nonzeroes                      0
   No of matrix inequal.            1
     detected matrix inq.           1
       linear                       1
       nonlinear                    0
       max. dimension              21
     detected normal inq.           0
       linear                       0
       nonlinear                    0
   Objective function          Linear

 --------------------------------------------------------------
  it|  objective |  optim  |   feas  |  compl  | pen min |inner
 --------------------------------------------------------------
   0  0.00000E+00  9.17E+01  1.91E+00  0.00E+00  2.00E+00   0
   1  1.53207E+02  6.26E-03  0.00E+00  1.50E+02  2.00E+00  13
   2  7.10956E+01  1.75E-02  0.00E+00  6.45E+01  9.04E-01   4
   3

We can now retrieve the completed matrix!

In [10]:
# Fill the missing elements of the matrix
idx = int(nr*(nr+1)/2 + nq*(nq+1)/2)
for i in range(nr):
    for j in range(nq):
        if Y[i][j] < 0.0:
            Y[i][j] = x[idx]
            idx += 1
            
np.set_printoptions(precision=1)
print(Y)

rk = np.linalg.matrix_rank(Y, tol=1.0e-05)
print('The rank of the filled matrix is:', rk)

[[0.5 0.3 0.2 0.2 0.4 0.4]
 [0.6 0.4 0.8 0.2 0.3 0.4]
 [0.4 0.3 0.8 0.  0.2 0.2]
 [0.8 0.2 0.3 0.4 0.3 0.4]
 [0.  0.4 0.2 0.  0.2 0.2]
 [0.4 0.1 0.2 0.2 0.1 0.2]
 [0.6 0.8 0.2 0.6 0.2 0.4]
 [0.1 0.1 0.2 0.  0.  0.1]
 [0.6 0.4 0.1 0.6 0.  0.3]
 [0.2 0.1 0.4 0.  0.1 0.1]
 [0.5 0.3 0.2 0.2 0.4 0.4]
 [0.7 0.4 0.3 0.  1.  0.8]
 [1.  0.3 0.2 0.5 0.5 0.6]
 [0.2 0.1 0.1 0.1 0.2 0.2]
 [0.6 0.3 0.2 0.4 0.2 0.3]]
The rank of the filled matrix is: 4


In [11]:
help(opt.handle_solve_pennon)

Help on function handle_solve_pennon in module naginterfaces.library.opt:

handle_solve_pennon(handle, x, inform, u=None, uc=None, ua=None, io_manager=None)
    Run the Pennon solver on a compatible problem initialized by
    ``handle_init`` and defined by other functions from the suite, such
    as, semidefinite programming (SDP) and SDP with bilinear matrix
    inequalities (BMI).
    
    Note: this function uses optional algorithmic parameters, see also:
    ``handle_opt_set``, ``handle_opt_get``.
    
    ``handle_solve_pennon`` is a solver from the NAG optimization
    modelling suite for problems such as, Quadratic Programming (QP),
    linear Semidefinite Programming (SDP) and semidefinite programming
    with bilinear matrix inequalities (BMI-SDP).
    
    For full information please refer to the NAG Library document for
    e04sv
    
    https://www.nag.com/numeric/nl/nagdoc_27.1/flhtml/e04/e04svf.html
    
    Parameters
    ----------
    handle : Handle
        The handl